In [2]:
!pip install monai

Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/share/pkg.7/python3/3.7.9/install/bin/python3.7 -m pip install --upgrade pip' command.


In [3]:
# First cell: Imports and Dataset Class
import os
import pandas as pd
import nibabel as nib
from pathlib import Path
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader
from monai.networks.nets import ViT
from monai.transforms import (
    Compose,
    LoadImage,
    Resize,
    ScaleIntensity,
    EnsureChannelFirst,
    RandRotate90,
    RandFlip,
)
from sklearn.model_selection import train_test_split
import torch.nn as nn

class HemorrhageDataset(Dataset):
    def __init__(self, base_dir, image_ids, transforms=None):
        self.base_dir = Path(base_dir)
        self.image_ids = image_ids
        self.transforms = transforms
        
    def __len__(self):
        return len(self.image_ids)
    
    def __getitem__(self, idx):
        img_id = self.image_ids[idx]
        folder_path = self.base_dir / f"ID_{img_id}"
        
        # Find the NIfTI file in the folder
        nifti_files = list(folder_path.glob('*.nii.gz'))
        if not nifti_files:
            raise FileNotFoundError(f"No NIfTI file found in {folder_path}")
        
        # Load the NIfTI file
        img = nib.load(str(nifti_files[0]))
        img_data = img.get_fdata()
        
        # Load and parse the CSV file
        csv_path = folder_path / 'hemorrhage_labels.csv'
        if not csv_path.exists():
            raise FileNotFoundError(f"Labels file not found: {csv_path}")
            
        labels_df = pd.read_csv(csv_path)
        
        # Extract labels (excluding the 'any' category)
        labels = labels_df.set_index('hemorrhage_type')['label']
        label_array = np.array([
            labels['epidural'],
            labels['intraparenchymal'],
            labels['intraventricular'],
            labels['subarachnoid'],
            labels['subdural']
        ], dtype=np.float32)
        
        if self.transforms:
            img_data = self.transforms(img_data)
        
        return img_data, label_array

In [4]:
# Second cell: Dataset Analysis Function
def analyze_dataset_dimensions(base_dir):
    """Analyze the dimensions of NIfTI files in the dataset"""
    folders = list(Path(base_dir).glob('ID_*'))
    dimensions = []
    
    for folder in folders[:10]:  # Analyze first 10 files as sample
        nifti_files = list(folder.glob('*.nii.gz'))
        if nifti_files:
            img = nib.load(str(nifti_files[0]))
            dimensions.append(img.shape)
            print(f"Folder: {folder.name}, Dimensions: {img.shape}")
    
    return dimensions

# Run the analysis
base_dir = "/projectnb/ec523kb/projects/hemorrhage-classification/stage_2_train_sorted_nifti_pruned"  # Replace with your data directory
dimensions = analyze_dataset_dimensions(base_dir)
print("\nSummary of dimensions found:", dimensions)

Folder: ID_15298ce049, Dimensions: (512, 512, 32)
Folder: ID_a97c63329f, Dimensions: (512, 512, 37)
Folder: ID_8f1cbd1653, Dimensions: (512, 512, 31)
Folder: ID_1a5778261b, Dimensions: (512, 512, 34)
Folder: ID_9f3504cb4e, Dimensions: (512, 512, 31)
Folder: ID_b9a3bdea0c, Dimensions: (512, 512, 36)
Folder: ID_b1ddc133e5, Dimensions: (512, 512, 28)
Folder: ID_fcb5498cfb, Dimensions: (512, 512, 28)
Folder: ID_014634e079, Dimensions: (512, 512, 32)
Folder: ID_ec52c05b93, Dimensions: (512, 512, 32)

Summary of dimensions found: [(512, 512, 32), (512, 512, 37), (512, 512, 31), (512, 512, 34), (512, 512, 31), (512, 512, 36), (512, 512, 28), (512, 512, 28), (512, 512, 32), (512, 512, 32)]


In [21]:
import os
import pandas as pd
import nibabel as nib
from pathlib import Path
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader
from monai.transforms import (
    Compose,
    EnsureChannelFirst,
    Resize,
    ScaleIntensity,
)
from sklearn.model_selection import train_test_split

class HemorrhageDataset(Dataset):
    def __init__(self, base_dir, image_ids, transforms=None):
        self.base_dir = Path(base_dir)
        self.image_ids = image_ids
        self.transforms = transforms
        
    def __len__(self):
        return len(self.image_ids)
    
    def __getitem__(self, idx):
        img_id = self.image_ids[idx]
        folder_path = self.base_dir / f"ID_{img_id}"
        
        # Find the NIfTI file in the folder
        nifti_files = list(folder_path.glob('*.nii.gz'))
        if not nifti_files:
            raise FileNotFoundError(f"No NIfTI file found in {folder_path}")
        
        # Load the NIfTI file
        img = nib.load(str(nifti_files[0]))
        img_data = img.get_fdata()
        
        # Convert to tensor and add channel dimension
        img_data = torch.from_numpy(img_data).float()
        img_data = img_data.unsqueeze(0)  # Add channel dimension
        
        if self.transforms:
            img_data = self.transforms(img_data)
        
        # Load and parse the CSV file
        csv_path = folder_path / 'hemorrhage_labels.csv'
        if not csv_path.exists():
            raise FileNotFoundError(f"Labels file not found: {csv_path}")
            
        labels_df = pd.read_csv(csv_path)
        
        # Extract labels (excluding the 'any' category)
        labels = labels_df.set_index('hemorrhage_type')['label']
        label_array = np.array([
            labels['epidural'],
            labels['intraparenchymal'],
            labels['intraventricular'],
            labels['subarachnoid'],
            labels['subdural']
        ], dtype=np.float32)
        
        label_tensor = torch.from_numpy(label_array).float()
        
        return img_data, label_tensor

def custom_collate(batch):
    """Simplified collate function since all images are same size"""
    images, labels = zip(*batch)
    images_tensor = torch.stack(images)
    labels_tensor = torch.stack(labels)
    return images_tensor, labels_tensor

def create_data_pipeline(base_dir, num_train=300, num_val=100):
    """Create data pipeline with train/val split"""
    
    # Get list of all image IDs
    folders = list(Path(base_dir).glob('ID_*'))
    image_ids = [folder.name.split('_')[1] for folder in folders]
    
    print(f"Found {len(image_ids)} total images")
    
    # Create train/val split
    train_ids, val_ids = train_test_split(
        image_ids, 
        train_size=num_train,
        test_size=num_val,
        random_state=42
    )
    
    print(f"Split into {len(train_ids)} training and {len(val_ids)} validation images")
    
    # Define transforms
    transforms = Compose([
        ScaleIntensity(),
        Resize((128, 128, 128), mode='trilinear')  # Add this line
    ])
    
    # Create datasets with transforms
    train_ds = HemorrhageDataset(base_dir, train_ids, transforms=transforms)
    val_ds = HemorrhageDataset(base_dir, val_ids, transforms=transforms)
    
    return train_ds, val_ds


# Create datasets and dataloaders
print("Creating data pipeline...")
train_ds, val_ds = create_data_pipeline(base_dir)

# Create data loaders with custom collate function
train_loader = DataLoader(
    train_ds, 
    batch_size=2, 
    shuffle=True, 
    num_workers=2,
    pin_memory=True if torch.cuda.is_available() else False,
    collate_fn=custom_collate
)

val_loader = DataLoader(
    val_ds, 
    batch_size=2, 
    shuffle=False, 
    num_workers=2,
    pin_memory=True if torch.cuda.is_available() else False,
    collate_fn=custom_collate
)

print("Data pipeline created successfully!")
print(f"Number of training batches: {len(train_loader)}")
print(f"Number of validation batches: {len(val_loader)}")

# Test the data loading
print("\nTesting data loading...")
try:
    # Get the first batch
    sample_batch = next(iter(train_loader))
    images, labels = sample_batch
    
    print("Successfully loaded a batch!")
    print(f"Batch image shape: {images.shape}")
    print(f"Batch labels shape: {labels.shape}")
    print(f"Image data type: {images.dtype}")
    print(f"Labels data type: {labels.dtype}")
    
    # Print value ranges
    print(f"\nImage value range: [{images.min():.3f}, {images.max():.3f}]")
    print(f"Labels value range: [{labels.min():.3f}, {labels.max():.3f}]")
    
except Exception as e:
    print(f"\nError during data loading: {str(e)}")
    print("\nTraceback:")
    import traceback
    traceback.print_exc()

Creating data pipeline...
Found 3784 total images
Split into 300 training and 100 validation images
Data pipeline created successfully!
Number of training batches: 150
Number of validation batches: 50

Testing data loading...
Successfully loaded a batch!
Batch image shape: torch.Size([2, 1, 128, 128, 128])
Batch labels shape: torch.Size([2, 5])
Image data type: torch.float32
Labels data type: torch.float32

Image value range: [0.000, 0.985]
Labels value range: [0.000, 1.000]


In [20]:
!pip install --user einops

You should consider upgrading via the '/share/pkg.7/python3/3.7.9/install/bin/python3.7 -m pip install --upgrade pip' command.


In [22]:
# Test the implementation
if __name__ == "__main__":
    base_dir = "/projectnb/ec523kb/projects/hemorrhage-classification/stage_2_train_sorted_nifti_pruned"
    
    try:
        # Create data loaders
        train_loader, val_loader = create_data_loaders(
            base_dir,
            batch_size=2,
            train_size=300,
            val_size=100
        )
        
        # Test a batch
        print("\nTesting batch loading...")
        batch = next(iter(train_loader))
        images, labels = batch
        
        print(f"Batch image shape: {images.shape}")
        print(f"Batch label shape: {labels.shape}")
        print(f"Image value range: [{images.min():.3f}, {images.max():.3f}]")
        
        print("\nData loaders created successfully!")
        print(f"Number of training batches: {len(train_loader)}")
        print(f"Number of validation batches: {len(val_loader)}")
        
        # Print label distribution in the batch
        print("\nLabel distribution in batch:")
        print("Labels sum:", labels.sum(dim=0))  # Sum for each class
        
    except Exception as e:
        print(f"\nError encountered: {str(e)}")
        print("\nTraceback:")
        import traceback
        traceback.print_exc()

Creating data loaders...
Total images found: 3784
Training images: 300
Validation images: 100

Testing batch loading...

Error encountered: Caught RuntimeError in DataLoader worker process 0.
Original Traceback (most recent call last):
  File "/share/pkg.7/pytorch/1.12.1/install/lib/SCC/../python3.7/site-packages/torch/utils/data/_utils/worker.py", line 302, in _worker_loop
    data = fetcher.fetch(index)
  File "/share/pkg.7/pytorch/1.12.1/install/lib/SCC/../python3.7/site-packages/torch/utils/data/_utils/fetch.py", line 52, in fetch
    return self.collate_fn(data)
  File "/share/pkg.7/pytorch/1.12.1/install/lib/SCC/../python3.7/site-packages/torch/utils/data/_utils/collate.py", line 175, in default_collate
    return [default_collate(samples) for samples in transposed]  # Backwards compatibility.
  File "/share/pkg.7/pytorch/1.12.1/install/lib/SCC/../python3.7/site-packages/torch/utils/data/_utils/collate.py", line 175, in <listcomp>
    return [default_collate(samples) for samples 

Traceback (most recent call last):
  File "<ipython-input-22-1b8e961f7e5d>", line 16, in <module>
    batch = next(iter(train_loader))
  File "/share/pkg.7/pytorch/1.12.1/install/lib/SCC/../python3.7/site-packages/torch/utils/data/dataloader.py", line 681, in __next__
    data = self._next_data()
  File "/share/pkg.7/pytorch/1.12.1/install/lib/SCC/../python3.7/site-packages/torch/utils/data/dataloader.py", line 1376, in _next_data
    return self._process_data(data)
  File "/share/pkg.7/pytorch/1.12.1/install/lib/SCC/../python3.7/site-packages/torch/utils/data/dataloader.py", line 1402, in _process_data
    data.reraise()
  File "/share/pkg.7/pytorch/1.12.1/install/lib/SCC/../python3.7/site-packages/torch/_utils.py", line 461, in reraise
    raise exception
RuntimeError: Caught RuntimeError in DataLoader worker process 0.
Original Traceback (most recent call last):
  File "/share/pkg.7/pytorch/1.12.1/install/lib/SCC/../python3.7/site-packages/torch/utils/data/_utils/worker.py", line 30

In [18]:
import torch
import torch.nn as nn
from monai.networks.nets import ViT

def create_model():
    """Create and configure the Vision Transformer model for hemorrhage classification"""
    model = ViT(
        in_channels=1,                  # Single channel CT scans
        img_size=(128, 128, 128),       # Our resized image dimensions
        patch_size=(16, 16, 16),        # Divide image into 8x8x8 patches
        hidden_size=512,                # Size of hidden layers
        mlp_dim=1024,                   # MLP dimension (2x hidden_size)
        num_layers=8,                   # Number of transformer layers
        num_heads=8,                    # Number of attention heads
        pos_embed="conv",               # Use conv position embedding
        classification=True,            # Enable classification mode
        num_classes=5,                  # Our 5 hemorrhage types
        spatial_dims=3,                 # 3D images
        dropout_rate=0.1                # Dropout for regularization
    )
    return model

def initialize_training(model, learning_rate=1e-4, weight_decay=1e-5):
    """Initialize optimizer and loss function"""
    optimizer = torch.optim.AdamW(
        model.parameters(),
        lr=learning_rate,
        weight_decay=weight_decay
    )
    criterion = nn.BCEWithLogitsLoss()  # Binary cross-entropy for multi-label
    return optimizer, criterion

# Test the model
if __name__ == "__main__":
    # Set device
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(f"Using device: {device}")
    
    try:
        # Create model
        model = create_model()
        model = model.to(device)
        
        # Initialize training components
        optimizer, criterion = initialize_training(model)
        
        # Print model summary
        print("\nModel created successfully!")
        print(f"Total parameters: {sum(p.numel() for p in model.parameters()):,}")
        
        # Test with a sample batch from our dataloader
        batch = next(iter(train_loader))
        images, labels = batch
        images = images.to(device)
        labels = labels.to(device)
        
        # Forward pass
        outputs, hidden_states = model(images)
        
        print("\nTest forward pass successful!")
        print(f"Input shape: {images.shape}")
        print(f"Output shape: {outputs.shape}")
        print(f"Number of hidden states: {len(hidden_states)}")
        
        # Calculate loss
        loss = criterion(outputs, labels)
        print(f"Test loss calculation: {loss.item():.4f}")
        
        print("\nMemory usage:")
        print(f"Memory allocated: {torch.cuda.memory_allocated()/1e9:.2f} GB")
        print(f"Memory cached: {torch.cuda.memory_reserved()/1e9:.2f} GB")
        
    except Exception as e:
        print(f"\nError encountered: {str(e)}")
        print("\nTraceback:")
        import traceback
        traceback.print_exc()

Using device: cuda

Model created successfully!
Total parameters: 19,173,893

Test forward pass successful!
Input shape: torch.Size([2, 1, 128, 128, 128])
Output shape: torch.Size([2, 5])
Number of hidden states: 8
Test loss calculation: 0.6978

Memory usage:
Memory allocated: 1.19 GB
Memory cached: 1.59 GB


In [20]:
import torch
import torch.nn as nn
from tqdm import tqdm
import numpy as np
from sklearn.metrics import roc_auc_score, average_precision_score
import time
from pathlib import Path
import copy

class AverageMeter:
    """Computes and stores the average and current value"""
    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

def calculate_metrics(labels, outputs):
    """Calculate metrics for multi-label classification"""
    # Convert outputs to probabilities
    probs = torch.sigmoid(outputs)
    
    # Move tensors to CPU and convert to numpy
    labels_np = labels.cpu().numpy()
    probs_np = probs.detach().cpu().numpy()
    
    # Calculate metrics
    try:
        # ROC AUC for each class
        roc_auc = roc_auc_score(labels_np, probs_np, average=None)
        mean_roc_auc = np.mean(roc_auc)
        
        # Average Precision for each class
        avg_precision = average_precision_score(labels_np, probs_np, average=None)
        mean_avg_precision = np.mean(avg_precision)
        
        return {
            'roc_auc': mean_roc_auc,
            'avg_precision': mean_avg_precision,
            'roc_auc_per_class': roc_auc,
            'avg_precision_per_class': avg_precision
        }
    except ValueError:
        return {
            'roc_auc': 0.0,
            'avg_precision': 0.0,
            'roc_auc_per_class': np.zeros(5),
            'avg_precision_per_class': np.zeros(5)
        }

def train_epoch(model, train_loader, criterion, optimizer, scheduler, device):
    """Train for one epoch"""
    model.train()
    losses = AverageMeter()
    all_labels = []
    all_outputs = []
    
    pbar = tqdm(train_loader, desc='Training')
    for images, labels in pbar:
        # Move data to device
        images = images.to(device)
        labels = labels.to(device)
        
        # Forward pass
        optimizer.zero_grad()
        outputs, _ = model(images)
        loss = criterion(outputs, labels)
        
        # Backward pass
        loss.backward()
        optimizer.step()
        
        # Update metrics
        losses.update(loss.item(), images.size(0))
        all_labels.append(labels)
        all_outputs.append(outputs)
        
        pbar.set_postfix({'Loss': f'{losses.avg:.4f}'})
    
    # Calculate epoch metrics
    all_labels = torch.cat(all_labels)
    all_outputs = torch.cat(all_outputs)
    metrics = calculate_metrics(all_labels, all_outputs)
    metrics['loss'] = losses.avg
    
    # Step the scheduler
    if scheduler is not None:
        scheduler.step()
    
    return metrics

def validate(model, val_loader, criterion, device):
    """Validate the model"""
    model.eval()
    losses = AverageMeter()
    all_labels = []
    all_outputs = []
    
    with torch.no_grad():
        pbar = tqdm(val_loader, desc='Validation')
        for images, labels in pbar:
            images = images.to(device)
            labels = labels.to(device)
            
            # Forward pass
            outputs, _ = model(images)
            loss = criterion(outputs, labels)
            
            # Update metrics
            losses.update(loss.item(), images.size(0))
            all_labels.append(labels)
            all_outputs.append(outputs)
            
            pbar.set_postfix({'Loss': f'{losses.avg:.4f}'})
    
    # Calculate metrics
    all_labels = torch.cat(all_labels)
    all_outputs = torch.cat(all_outputs)
    metrics = calculate_metrics(all_labels, all_outputs)
    metrics['loss'] = losses.avg
    
    return metrics

def train_model(model, train_loader, val_loader, criterion, optimizer, 
                num_epochs, device, checkpoint_dir='checkpoints'):
    """Complete training pipeline"""
    # Create checkpoint directory
    checkpoint_dir = Path(checkpoint_dir)
    checkpoint_dir.mkdir(exist_ok=True)
    
    # Initialize learning rate scheduler
    scheduler = torch.optim.lr_scheduler.OneCycleLR(
        optimizer,
        max_lr=1e-3,
        epochs=num_epochs,
        steps_per_epoch=len(train_loader),
        pct_start=0.3,  # Spend 30% of training warming up
        div_factor=25.0,  # Initial lr = max_lr/25
        final_div_factor=10000.0  # Final lr = max_lr/10000
    )
    
    best_val_loss = float('inf')
    best_val_auc = 0.0
    best_model = None
    
    # Training loop
    for epoch in range(num_epochs):
        print(f'\nEpoch {epoch+1}/{num_epochs}')
        print('-' * 20)
        
        # Train
        train_metrics = train_epoch(model, train_loader, criterion, optimizer, scheduler, device)
        print('\nTraining Metrics:')
        for k, v in train_metrics.items():
            if isinstance(v, np.ndarray):
                print(f'{k}: {v}')
            else:
                print(f'{k}: {v:.4f}')
        
        # Validate
        val_metrics = validate(model, val_loader, criterion, device)
        print('\nValidation Metrics:')
        for k, v in val_metrics.items():
            if isinstance(v, np.ndarray):
                print(f'{k}: {v}')
            else:
                print(f'{k}: {v:.4f}')
        
        # Save best model by validation loss
        if val_metrics['loss'] < best_val_loss:
            best_val_loss = val_metrics['loss']
            torch.save({
                'epoch': epoch,
                'model_state_dict': model.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),
                'scheduler_state_dict': scheduler.state_dict(),
                'metrics': val_metrics,
            }, checkpoint_dir / 'best_loss_model.pth')
        
        # Save best model by ROC AUC
        if val_metrics['roc_auc'] > best_val_auc:
            best_val_auc = val_metrics['roc_auc']
            best_model = copy.deepcopy(model.state_dict())
            torch.save({
                'epoch': epoch,
                'model_state_dict': model.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),
                'scheduler_state_dict': scheduler.state_dict(),
                'metrics': val_metrics,
            }, checkpoint_dir / 'best_auc_model.pth')
        
        # Save periodic checkpoint
        if (epoch + 1) % 5 == 0:
            torch.save({
                'epoch': epoch,
                'model_state_dict': model.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),
                'scheduler_state_dict': scheduler.state_dict(),
                'metrics': val_metrics,
            }, checkpoint_dir / f'epoch_{epoch+1}_checkpoint.pth')
    
    return best_model

# Training configuration and execution
if __name__ == "__main__":
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    num_epochs = 50
    checkpoint_dir = 'hemorrhage_checkpoints'
    
    # Train the model
    best_model_state = train_model(
        model=model,
        train_loader=train_loader,
        val_loader=val_loader,
        criterion=criterion,
        optimizer=optimizer,
        num_epochs=num_epochs,
        device=device,
        checkpoint_dir=checkpoint_dir
    )

Training:   0%|          | 0/150 [00:00<?, ?it/s]


Epoch 1/50
--------------------


Validation:   0%|          | 0/50 [00:00<?, ?it/s]


Training Metrics:
roc_auc: 0.4831
avg_precision: 0.0888
roc_auc_per_class: [0.33501684 0.47422138 0.50495846 0.50629572 0.5952067 ]
avg_precision_per_class: [0.00956839 0.10950379 0.0498298  0.11234712 0.16273345]
loss: 0.3960


Validation:  76%|███████▌  | 38/50 [00:07<00:02,  5.21it/s, Loss=0.3896]


RuntimeError: Caught RuntimeError in DataLoader worker process 2.
Original Traceback (most recent call last):
  File "/usr4/ec523/ibhattac/.local/lib/python3.7/site-packages/monai/transforms/transform.py", line 102, in apply_transform
    return _apply_transform(transform, data, unpack_items)
  File "/usr4/ec523/ibhattac/.local/lib/python3.7/site-packages/monai/transforms/transform.py", line 66, in _apply_transform
    return transform(parameters)
  File "/usr4/ec523/ibhattac/.local/lib/python3.7/site-packages/monai/transforms/croppad/array.py", line 1322, in __call__
    ret = self.padder(self.cropper(img), mode=mode, **pad_kwargs)
  File "/usr4/ec523/ibhattac/.local/lib/python3.7/site-packages/monai/transforms/croppad/array.py", line 543, in __call__
    return super().__call__(img=img, slices=self.compute_slices(img.shape[1:]))
  File "/usr4/ec523/ibhattac/.local/lib/python3.7/site-packages/monai/transforms/croppad/array.py", line 533, in compute_slices
    roi_size = fall_back_tupl